<a href="https://colab.research.google.com/github/google-research/tapas/blob/master/notebooks/sqa_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Running a Tapas fine-tuned checkpoint


In [ ]:
! pip install tapas-table-parsing

     |████████████████████████████████| 195 kB 5.2 MB/s 
     |████████████████████████████████| 10.1 MB 38.0 MB/s 
     |████████████████████████████████| 516.4 MB 18 kB/s 
     |████████████████████████████████| 1.5 MB 26.1 MB/s 
     |████████████████████████████████| 3.5 MB 28.3 MB/s 
     |████████████████████████████████| 7.1 MB 47.0 MB/s 
     |████████████████████████████████| 352 kB 47.9 MB/s 
     |████████████████████████████████| 711 kB 47.3 MB/s 
     |████████████████████████████████| 58 kB 4.2 MB/s 
     |████████████████████████████████| 3.5 MB 37.5 MB/s 
     |████████████████████████████████| 218 kB 54.4 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 63.1 MB 37 kB/s 
     |████████████████████████████████| 1.2 MB 50.9 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 151 kB 52.2 MB/s 
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |█████████████████

# Fetch models fom Google Storage

Next we can get pretrained checkpoint from Google Storage. For the sake of speed, this is base sized model trained on [SQA](https://www.microsoft.com/en-us/download/details.aspx?id=54253). Note that best results in the paper were obtained with a large model, with 24 layers instead of 12.

In [ ]:

! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip

Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
/ [1 files][  1.0 GiB/  1.0 GiB]   44.4 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
   creating: tapas_sqa_base/
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


# Imports

In [ ]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [ ]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

# Load checkpoint for prediction

In [ ]:
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# access the postgresql server
conn = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")
cursor = conn.cursor()

In [ ]:
sql = '''
DROP TABLE IF EXISTS da_category_ids;
CREATE TEMP TABLE da_category_ids AS (
(SELECT data_officerallegation.id,data_allegationcategory.allegation_name,data_allegationcategory.category
FROM data_officerallegation
join data_allegationcategory on data_officerallegation.allegation_category_id = data_allegationcategory.id
WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
OR data_allegationcategory.category_code IN ('024', '003', '003A', '003B', '003C', '003D', '003E')));

SELECT gender,race,birth_year,first_name || ' ' || last_name officer_name,allegation_count,sustained_count,current_salary
FROM data_officer
join da_category_ids on data_officer.id=da_category_ids.id
where data_officer.current_salary IS NOT NULL and allegation_count>2 and sustained_count>1 order by officer_name desc;
'''

In [ ]:
cursor.execute(sql)
colnames = ['gender', 'race', 'birth year', 'officer name', 'allegation count', 'sustained count', 'current salary']
edges = cursor.fetchall()
res = [[str(x) for x in list(ele)] for ele in edges]
res.insert(0, colnames)
print(res)


[['gender', 'race', 'birth year', 'officer name', 'allegation count', 'sustained count', 'current salary'], ['M', 'Hispanic', '1952', 'Wilman Dones', '23', '6', '82008'], ['M', 'Black', '1963', 'Vernard Ross', '23', '3', '107988'], ['M', 'White', '1957', 'Thomas Motzny', '31', '2', '102978'], ['M', 'White', '1949', 'Thomas Biggane', '30', '2', '82878'], ['M', 'White', '1969', 'Steven Nowicki', '27', '3', '111474'], ['M', 'White', '1970', 'Steven Bechina', '43', '4', '107988'], ['M', 'White', '1948', 'Ronald Blake', '6', '2', '83604'], ['M', 'Black', '1948', 'Rollins Johnson', '6', '2', '88260'], ['M', 'Hispanic', '1985', 'Roger Farias', '14', '2', '84054'], ['M', 'Black', '1964', 'Ricky Bean', '11', '3', '100980'], ['M', 'White', '1961', 'Raymond Gadomski', '17', '3', '96060'], ['M', 'Black', '1953', 'Prentiss Jackson', '15', '5', '102978'], ['M', 'White', '1962', 'Philip Paluch', '56', '2', '111474'], ['F', 'Black', '1950', 'Paularie Draine', '7', '2', '78012'], ['F', 'Black', '1948',

In [ ]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [ ]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  answers=[]
  table = res
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers.append(', '.join([table[row + 1][col] for row, col in coordinates]))
      position = int(row['position'])
  return answers

Get sql column data

In [ ]:
officer_name=[el[3] for el in res][1:]
print(officer_name)

['Wilman Dones', 'Vernard Ross', 'Thomas Motzny', 'Thomas Biggane', 'Steven Nowicki', 'Steven Bechina', 'Ronald Blake', 'Rollins Johnson', 'Roger Farias', 'Ricky Bean', 'Raymond Gadomski', 'Prentiss Jackson', 'Philip Paluch', 'Paularie Draine', 'Patricia Ballentine', 'Orlando Fonseca', 'Nicholas Dimaggio', 'Michael Overstreet', 'Marvin Randolph', 'Marshall Pufundt', 'Marco Johnson', 'Luis Lopez', 'Lisa William-Handley', 'Linda Brumfield', 'Larry Dotson', 'Lakisa Anderson', 'Kimberly Hill', 'Kevin Keyes', 'Joseph Thompson', 'Joseph Battaglia', 'John Brownridge', 'Jesus Avila', 'Jeanetta Brown Cunningha', 'Jacklyn Mueller', 'Irwin Negron', 'Haki Akintunde', 'Frederick Anthony', 'Emil Bux', 'Donald Banks', 'Dennis Peca', 'Brian Burton', 'Anthony Singleton', 'Anita Ashton', 'Alvin Ward', 'Alvin Greenup']


In [ ]:
race=[]
for x in officer_name:
  race.append("What is the race of "+x+"?")
print(race)

['What is the race of Wilman Dones?', 'What is the race of Vernard Ross?', 'What is the race of Thomas Motzny?', 'What is the race of Thomas Biggane?', 'What is the race of Steven Nowicki?', 'What is the race of Steven Bechina?', 'What is the race of Ronald Blake?', 'What is the race of Rollins Johnson?', 'What is the race of Roger Farias?', 'What is the race of Ricky Bean?', 'What is the race of Raymond Gadomski?', 'What is the race of Prentiss Jackson?', 'What is the race of Philip Paluch?', 'What is the race of Paularie Draine?', 'What is the race of Patricia Ballentine?', 'What is the race of Orlando Fonseca?', 'What is the race of Nicholas Dimaggio?', 'What is the race of Michael Overstreet?', 'What is the race of Marvin Randolph?', 'What is the race of Marshall Pufundt?', 'What is the race of Marco Johnson?', 'What is the race of Luis Lopez?', 'What is the race of Lisa William-Handley?', 'What is the race of Linda Brumfield?', 'What is the race of Larry Dotson?', 'What is the rac

In [ ]:
gender=[]
for x in officer_name:
  gender.append("What is the gender of "+x+"?")
print(gender)

['What is the gender of Wilman Dones?', 'What is the gender of Vernard Ross?', 'What is the gender of Thomas Motzny?', 'What is the gender of Thomas Biggane?', 'What is the gender of Steven Nowicki?', 'What is the gender of Steven Bechina?', 'What is the gender of Ronald Blake?', 'What is the gender of Rollins Johnson?', 'What is the gender of Roger Farias?', 'What is the gender of Ricky Bean?', 'What is the gender of Raymond Gadomski?', 'What is the gender of Prentiss Jackson?', 'What is the gender of Philip Paluch?', 'What is the gender of Paularie Draine?', 'What is the gender of Patricia Ballentine?', 'What is the gender of Orlando Fonseca?', 'What is the gender of Nicholas Dimaggio?', 'What is the gender of Michael Overstreet?', 'What is the gender of Marvin Randolph?', 'What is the gender of Marshall Pufundt?', 'What is the gender of Marco Johnson?', 'What is the gender of Luis Lopez?', 'What is the gender of Lisa William-Handley?', 'What is the gender of Linda Brumfield?', 'What

In [ ]:
birth_year=[]
for x in officer_name:
  birth_year.append("What is the birth year of "+x+"?")
print(birth_year)

['What is the birth year of Wilman Dones?', 'What is the birth year of Vernard Ross?', 'What is the birth year of Thomas Motzny?', 'What is the birth year of Thomas Biggane?', 'What is the birth year of Steven Nowicki?', 'What is the birth year of Steven Bechina?', 'What is the birth year of Ronald Blake?', 'What is the birth year of Rollins Johnson?', 'What is the birth year of Roger Farias?', 'What is the birth year of Ricky Bean?', 'What is the birth year of Raymond Gadomski?', 'What is the birth year of Prentiss Jackson?', 'What is the birth year of Philip Paluch?', 'What is the birth year of Paularie Draine?', 'What is the birth year of Patricia Ballentine?', 'What is the birth year of Orlando Fonseca?', 'What is the birth year of Nicholas Dimaggio?', 'What is the birth year of Michael Overstreet?', 'What is the birth year of Marvin Randolph?', 'What is the birth year of Marshall Pufundt?', 'What is the birth year of Marco Johnson?', 'What is the birth year of Luis Lopez?', 'What 

In [ ]:
allegation_count=[]
for x in officer_name:
  allegation_count.append("What is the allegation count of "+x+"?")
print(allegation_count)

['What is the allegation count of Wilman Dones?', 'What is the allegation count of Vernard Ross?', 'What is the allegation count of Thomas Motzny?', 'What is the allegation count of Thomas Biggane?', 'What is the allegation count of Steven Nowicki?', 'What is the allegation count of Steven Bechina?', 'What is the allegation count of Ronald Blake?', 'What is the allegation count of Rollins Johnson?', 'What is the allegation count of Roger Farias?', 'What is the allegation count of Ricky Bean?', 'What is the allegation count of Raymond Gadomski?', 'What is the allegation count of Prentiss Jackson?', 'What is the allegation count of Philip Paluch?', 'What is the allegation count of Paularie Draine?', 'What is the allegation count of Patricia Ballentine?', 'What is the allegation count of Orlando Fonseca?', 'What is the allegation count of Nicholas Dimaggio?', 'What is the allegation count of Michael Overstreet?', 'What is the allegation count of Marvin Randolph?', 'What is the allegation 

In [ ]:
sustained_count=[]
for x in officer_name:
  sustained_count.append("What is the sustained count of "+x+"?")
print(sustained_count)

['What is the sustained count of Wilman Dones?', 'What is the sustained count of Vernard Ross?', 'What is the sustained count of Thomas Motzny?', 'What is the sustained count of Thomas Biggane?', 'What is the sustained count of Steven Nowicki?', 'What is the sustained count of Steven Bechina?', 'What is the sustained count of Ronald Blake?', 'What is the sustained count of Rollins Johnson?', 'What is the sustained count of Roger Farias?', 'What is the sustained count of Ricky Bean?', 'What is the sustained count of Raymond Gadomski?', 'What is the sustained count of Prentiss Jackson?', 'What is the sustained count of Philip Paluch?', 'What is the sustained count of Paularie Draine?', 'What is the sustained count of Patricia Ballentine?', 'What is the sustained count of Orlando Fonseca?', 'What is the sustained count of Nicholas Dimaggio?', 'What is the sustained count of Michael Overstreet?', 'What is the sustained count of Marvin Randolph?', 'What is the sustained count of Marshall Pu

In [ ]:
current_salary=[]
for x in officer_name:
  current_salary.append("What is the current salary of "+x+"?")
print(current_salary)

['What is the current salary of Wilman Dones?', 'What is the current salary of Vernard Ross?', 'What is the current salary of Thomas Motzny?', 'What is the current salary of Thomas Biggane?', 'What is the current salary of Steven Nowicki?', 'What is the current salary of Steven Bechina?', 'What is the current salary of Ronald Blake?', 'What is the current salary of Rollins Johnson?', 'What is the current salary of Roger Farias?', 'What is the current salary of Ricky Bean?', 'What is the current salary of Raymond Gadomski?', 'What is the current salary of Prentiss Jackson?', 'What is the current salary of Philip Paluch?', 'What is the current salary of Paularie Draine?', 'What is the current salary of Patricia Ballentine?', 'What is the current salary of Orlando Fonseca?', 'What is the current salary of Nicholas Dimaggio?', 'What is the current salary of Michael Overstreet?', 'What is the current salary of Marvin Randolph?', 'What is the current salary of Marshall Pufundt?', 'What is th

# **Choose your column name in select_col**
```

```



In [ ]:
select_col = 'birth_year'
predict_col_list=[gender,race,birth_year,allegation_count,sustained_count,current_salary]
predict_col_str=['gender','race','birth_year','allegation_count','sustained_count','current_salary']

for x in range(0,len(predict_col_list)):
  if predict_col_str[x]==select_col:
    col_num=x
    col_name=predict_col_list[x]
    print(col_name[0])
print(col_num,col_name)

What is the birth year of Wilman Dones?
2 ['What is the birth year of Wilman Dones?', 'What is the birth year of Vernard Ross?', 'What is the birth year of Thomas Motzny?', 'What is the birth year of Thomas Biggane?', 'What is the birth year of Steven Nowicki?', 'What is the birth year of Steven Bechina?', 'What is the birth year of Ronald Blake?', 'What is the birth year of Rollins Johnson?', 'What is the birth year of Roger Farias?', 'What is the birth year of Ricky Bean?', 'What is the birth year of Raymond Gadomski?', 'What is the birth year of Prentiss Jackson?', 'What is the birth year of Philip Paluch?', 'What is the birth year of Paularie Draine?', 'What is the birth year of Patricia Ballentine?', 'What is the birth year of Orlando Fonseca?', 'What is the birth year of Nicholas Dimaggio?', 'What is the birth year of Michael Overstreet?', 'What is the birth year of Marvin Randolph?', 'What is the birth year of Marshall Pufundt?', 'What is the birth year of Marco Johnson?', 'What

In [ ]:
column_number=col_num
data_query=[el[column_number] for el in res][1:]
print(data_query)

['1952', '1963', '1957', '1949', '1969', '1970', '1948', '1948', '1985', '1964', '1961', '1953', '1962', '1950', '1948', '1971', '1955', '1951', '1973', '1947', '1959', '1967', '1961', '1963', '1956', '1970', '1964', '1960', '1979', '1961', '1957', '1959', '1965', '1981', '1965', '1968', '1968', '1963', '1959', '1954', '1970', '1962', '1943', '1949', '1963']


In [ ]:
print(col_name[0])

What is the birth year of Wilman Dones?


# Predict

In [ ]:
result = predict('', col_name)

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


In [ ]:
result

['1979, 1952',
 '1963',
 '1957',
 '1957',
 '1969, 1970',
 '1969',
 '1968',
 '1948, 1943',
 '1985',
 '1964',
 '1956, 1961',
 '1953',
 '1962',
 '1950',
 '1948',
 '1971',
 '1955',
 '1951',
 '1973',
 '1947',
 '1959',
 '1967',
 '1961',
 '1963',
 '1956, 1961',
 '1970',
 '1948, 1964',
 '1960, 1973',
 '1973',
 '1961',
 '1985',
 '1985',
 '1949, 1948',
 '1968',
 '1968',
 '1968, 1948',
 '1968',
 '1968, 1985',
 '1985',
 '1985',
 '1985',
 '1949',
 '',
 '',
 '1968, 1963, 1943']

In [ ]:
count=0
for x in range(0,len(result)):
  # print(result[x],data_query[x])
  if result[x]==data_query[x]:
    count=count+1
  else:
    a=result[x].split(" ")
    try: 
      if a[1]:
        count=count+1
    except:
      pass
print(count,len(result))
print("Accuracy = ","{:.2f}".format(count/len(result)*100))

31 45
Accuracy =  68.89
